# ImSpiRE: <ins>Im</ins>age-aided <ins>Sp</ins>at<ins>i</ins>al <ins>R</ins>esolution <ins>E</ins>nhancement

## This is a tutorial written using Jupyter Notebook.

### Step 1. ImSpiRE installation following the [tutorial](https://github.com/Yizhi-Zhang/ImSpiRE).

### Step 2. Input preparation

ImSpiRE utilizes the count file in tab-delimited format or hierarchical-data format (HDF5 or H5) and the image file in TIFF format, as well as a file containing spot coordinates as input. 

We provided a small [test dataset](https://github.com/Yizhi-Zhang/ImSpiRE/tree/master/test/test_data) containing the raw count matrix, image and spot coordinates. A CellProfiler pipeline is also included in the test dataset for use if required.

### Step 3. Operation of ImSpiRE

### 3.1 Load required packages

In [1]:
import imspire_object as imspire
import pandas as pd
import numpy as np
import scanpy as sc

### 3.2 Custom parameters

In [2]:
imspire_param=imspire.ImSpiRE_Parameters()

In [3]:
imspire_param.BasicParam_InputCountFile="test_data/count_matrix.tsv"
imspire_param.BasicParam_InputDir="test_data/"
imspire_param.BasicParam_InputImageFile="test_data/image.tif"
imspire_param.BasicParam_PlatForm="ST"
imspire_param.BasicParam_Mode=2
imspire_param.BasicParam_OutputName="test_output"
imspire_param.BasicParam_Overwriting=True
imspire_param.CellProfilerParam_Pipeline="test_data/Cellprofiler_Pipeline_HE.cppipe"
imspire_param.BasicParam_Verbose=True

### 3.3 Run ImSpiRE

#### 3.3.1 step by step

In [4]:
imspire.create_folder(imspire_param.BasicParam_OutputDir,
                      imspire_param.BasicParam_OutputName,
                      imspire_param.BasicParam_Overwriting)

In [5]:
imdata=imspire.ImSpiRE_Data()
imdata.read_ST(imspire_param.BasicParam_InputDir, count_file=imspire_param.BasicParam_InputCountFile)

In [6]:
## optional
imdata.preprocess(min_counts=imspire_param.Threshold_MinCounts, 
                  max_counts=imspire_param.Threshold_MaxCounts, 
                  pct_counts_mt=imspire_param.Threshold_MitoPercent, 
                  min_cells=imspire_param.Threshold_MinSpot)

In [7]:
im=imspire.ImSpiRE_HE_Image(imspire_param.BasicParam_InputImageFile,
                            imspire_param.BasicParam_PlatForm,
                            imspire_param.BasicParam_OutputDir,
                            imspire_param.BasicParam_OutputName,
                            imspire_param.FeatureParam_IterCount)

In [8]:
spot_image_output_path=f"{imspire_param.BasicParam_OutputDir}/{imspire_param.BasicParam_OutputName}/ImageResults/SpotImage"
im.segment_spot_image(pos_in_tissue_filter=imdata.pos_in_tissue_filter,
                      output_path=spot_image_output_path,
                      crop_size=imspire_param.ImageParam_CropSize)

In [9]:
patch_image_output_path=f"{imspire_param.BasicParam_OutputDir}/{imspire_param.BasicParam_OutputName}/ImageResults/PatchImage"
im.generate_patch_locations_2(pos_in_tissue=imdata.pos_in_tissue_filter,
                              dist=imspire_param.ImageParam_PatchDist)
im.segment_patch_image(patch_in_tissue=im.patch_in_tissue, 
                       output_path=patch_image_output_path, 
                       crop_size=imspire_param.ImageParam_CropSize)

In [10]:
spot_cp=imspire.ImSpiRE_HE_CellProfiler(image_file_path=f"{spot_image_output_path}", 
                                        cellprofiler_pipeline=imspire_param.CellProfilerParam_Pipeline)
patch_cp=imspire.ImSpiRE_HE_CellProfiler(image_file_path=f"{patch_image_output_path}", 
                                         cellprofiler_pipeline=imspire_param.CellProfilerParam_Pipeline)

In [11]:
spot_feature_output_path=f"{imspire_param.BasicParam_OutputDir}/{imspire_param.BasicParam_OutputName}/FeatureResults/SpotFeature"
spot_cp.compute_image_features(output_path=spot_feature_output_path,
                               number_of_kernels=imspire_param.CellProfilerParam_KernelNumber)
spot_cp.filter_image_features(output_path=spot_feature_output_path)

Experiment-wide values for mean threshold, etc calculated by MeasureImageQuality may be incorrect if the run is split into subsets of images.
Experiment-wide values for mean threshold, etc calculated by MeasureImageQuality may be incorrect if the run is split into subsets of images.
Times reported are CPU and Wall-clock times for each module
Thu Feb  9 10:08:30 2023: Image # 67, module Images # 1: CPU_time = 0.00 secs, Wall_time = 0.00 secs
Thu Feb  9 10:08:30 2023: Image # 67, module Metadata # 2: CPU_time = 0.00 secs, Wall_time = 0.00 secs
...
Experiment-wide values for mean threshold, etc calculated by MeasureImageQuality may be incorrect if the run is split into subsets of images.
Times reported are CPU and Wall-clock times for each module
Thu Feb  9 10:08:30 2023: Image # 12, module Images # 1: CPU_time = 0.00 secs, Wall_time = 0.00 secs
Thu Feb  9 10:08:30 2023: Image # 12, module Metadata # 2: CPU_time = 0.00 secs, Wall_time = 0.00 secs
Thu Feb  9 10:08:30 2023: Image # 67, modu

In [12]:
patch_feature_output_path=f"{imspire_param.BasicParam_OutputDir}/{imspire_param.BasicParam_OutputName}/FeatureResults/PatchFeature"
patch_cp.compute_image_features(output_path=patch_feature_output_path,
                                number_of_kernels=imspire_param.CellProfilerParam_KernelNumber)
patch_cp.filter_image_features(output_path=patch_feature_output_path)

Experiment-wide values for mean threshold, etc calculated by MeasureImageQuality may be incorrect if the run is split into subsets of images.
Given the large number of image sets, you may want to consider using ExportToDatabase as opposed to ExportToSpreadsheet.
Experiment-wide values for mean threshold, etc calculated by MeasureImageQuality may be incorrect if the run is split into subsets of images.
Times reported are CPU and Wall-clock times for each module
Thu Feb  9 10:09:00 2023: Image # 622, module Images # 1: CPU_time = 0.00 secs, Wall_time = 0.00 secs
...
Experiment-wide values for mean threshold, etc calculated by MeasureImageQuality may be incorrect if the run is split into subsets of images.
Times reported are CPU and Wall-clock times for each module
Thu Feb  9 10:09:01 2023: Image # 1, module Images # 1: CPU_time = 0.00 secs, Wall_time = 0.00 secs
Thu Feb  9 10:09:01 2023: Image # 1, module Metadata # 2: CPU_time = 0.00 secs, Wall_time = 0.00 secs
Thu Feb  9 10:09:00 2023:

Thu Feb  9 10:13:01 2023: Image # 1863, module Groups # 4: CPU_time = 0.00 secs, Wall_time = 0.00 secs
Thu Feb  9 10:13:01 2023: Image # 1863, module MeasureImageIntensity # 5: CPU_time = 0.03 secs, Wall_time = 0.03 secs
Thu Feb  9 10:13:01 2023: Image # 1449, module MeasureImageQuality # 6: CPU_time = 0.05 secs, Wall_time = 0.05 secs
Thu Feb  9 10:13:01 2023: Image # 1863, module MeasureImageQuality # 6: CPU_time = 0.07 secs, Wall_time = 0.14 secs
Thu Feb  9 10:13:01 2023: Image # 1449, module MeasureTexture # 7: CPU_time = 4.65 secs, Wall_time = 0.38 secs
Thu Feb  9 10:13:02 2023: Image # 1449, module ExportToSpreadsheet # 8: CPU_time = 0.00 secs, Wall_time = 0.00 secs
Thu Feb  9 10:13:02 2023: Image # 1449, module CreateBatchFiles # 9: CPU_time = 0.00 secs, Wall_time = 0.00 secs
Thu Feb  9 10:13:01 2023: Image # 1863, module MeasureTexture # 7: CPU_time = 5.99 secs, Wall_time = 0.33 secs
Thu Feb  9 10:13:02 2023: Image # 1863, module ExportToSpreadsheet # 8: CPU_time = 0.00 secs, Wa

In [13]:
spot_features=spot_cp.image_features.loc[imdata.pos_in_tissue_filter.index,]

patch_cp.image_features.index=patch_cp.image_features.index.astype('int')
patch_features=patch_cp.image_features.sort_index()
patch_features.index=list(range(patch_features.shape[0]))

spot_features.to_csv(f"{spot_feature_output_path}/Image_Features_Image_filter.txt", sep = "\t")
patch_features.to_csv(f"{patch_feature_output_path}/Image_Features_Image_filter.txt", sep = "\t")

In [14]:
spot_locations=np.array(imdata.pos_in_tissue_filter.loc[:,["pxl_row_in_fullres","pxl_col_in_fullres"]])
patch_locations=np.array(im.patch_in_tissue.loc[:,["pxl_row","pxl_col"]])

In [15]:
spot_feature=pd.read_csv(f"{spot_feature_output_path}/Image_Features_Image_filter.txt",sep="\t",index_col=0)
patch_feature=pd.read_csv(f"{patch_feature_output_path}/Image_Features_Image_filter.txt",sep="\t",index_col=0)

## make sure that spot features and patch features have the same dimension
spot_feature=spot_feature.dropna(axis=1)
patch_feature=patch_feature.dropna(axis=1)
commom_feature=list(set(spot_feature.columns).intersection(set(patch_feature.columns)))
spot_feature = spot_feature.loc[:,commom_feature]
patch_feature = patch_feature.loc[:,commom_feature]

spot_feature=np.array(spot_feature.loc[imdata.pos_in_tissue_filter.index,])
patch_feature=np.array(patch_feature.sort_index())

In [16]:
exp_data=imdata.adata.to_df()
exp_data=exp_data.loc[imdata.pos_in_tissue_filter.index,]

In [17]:
ot_solver=imspire.ImSpiRE_OT_Solver(spot_locations,patch_locations,
                                    spot_image_features=spot_feature,
                                    patch_image_features=patch_feature,
                                    spot_gene_expression=exp_data,
                                    random_state=imspire_param.BasicParam_RandomState)

In [18]:
ot_solver.setup_cost_matrices(alpha=imspire_param.OptimalTransportParam_Alpha,
                              num_neighbors=imspire_param.OptimalTransportParam_NumNeighbors)
ot_solver.solve_OT(beta=imspire_param.OptimalTransportParam_Beta, 
                   epsilon=imspire_param.OptimalTransportParam_Epsilon,
                   numItermax=imspire_param.OptimalTransportParam_NumIterMax,
                   verbose=imspire_param.BasicParam_Verbose)

It.  |Loss        |Relative loss|Absolute loss
------------------------------------------------
    0|1.867228e-01|0.000000e+00|0.000000e+00
    1|5.168294e-02|2.612853e+00|1.350399e-01
    2|4.871671e-02|6.088732e-02|2.966230e-03
    3|4.852903e-02|3.867217e-03|1.876723e-04
    4|4.847715e-02|1.070174e-03|5.187898e-05
    5|4.845565e-02|4.438386e-04|2.150649e-05
    6|4.844736e-02|1.711045e-04|8.289561e-06
    7|4.844367e-02|7.620973e-05|3.691879e-06
    8|4.844196e-02|3.522179e-05|1.706213e-06
    9|4.844115e-02|1.667971e-05|8.079845e-07
   10|4.844076e-02|8.140793e-06|3.943462e-07


In [19]:
exp_data_hr=imspire.compute_high_resolution_expression_profiles(exp_data,ot_solver.T)

In [20]:
## output results
im.patch_in_tissue.index=list(range(im.patch_in_tissue.shape[0]))
im.patch_in_tissue.to_csv(f"{imspire_param.BasicParam_OutputDir}/{imspire_param.BasicParam_OutputName}/{imspire_param.BasicParam_OutputName}_PatchLocations.txt", sep = "\t")

adata_hr=sc.AnnData(exp_data_hr)
adata_hr.write_h5ad(f"{imspire_param.BasicParam_OutputDir}/{imspire_param.BasicParam_OutputName}/{imspire_param.BasicParam_OutputName}_ResolutionEnhancementResult.h5ad")

np.save(f"{imspire_param.BasicParam_OutputDir}/{imspire_param.BasicParam_OutputName}/SupplementaryResults/ot_M_alpha{imspire_param.OptimalTransportParam_Alpha}_beta{imspire_param.OptimalTransportParam_Beta}_epsilon{imspire_param.OptimalTransportParam_Epsilon}.npy",ot_solver.M)
np.save(f"{imspire_param.BasicParam_OutputDir}/{imspire_param.BasicParam_OutputName}/SupplementaryResults/ot_C1_alpha{imspire_param.OptimalTransportParam_Alpha}_beta{imspire_param.OptimalTransportParam_Beta}_epsilon{imspire_param.OptimalTransportParam_Epsilon}.npy",ot_solver.C1)
np.save(f"{imspire_param.BasicParam_OutputDir}/{imspire_param.BasicParam_OutputName}/SupplementaryResults/ot_C2_alpha{imspire_param.OptimalTransportParam_Alpha}_beta{imspire_param.OptimalTransportParam_Beta}_epsilon{imspire_param.OptimalTransportParam_Epsilon}.npy",ot_solver.C2)
np.save(f"{imspire_param.BasicParam_OutputDir}/{imspire_param.BasicParam_OutputName}/SupplementaryResults/ot_T_alpha{imspire_param.OptimalTransportParam_Alpha}_beta{imspire_param.OptimalTransportParam_Beta}_epsilon{imspire_param.OptimalTransportParam_Epsilon}.npy",ot_solver.T)

#### 3.3.2 one step

In [21]:
imspire_run=imspire.ImSpiRE(imspire_param)

In [22]:
imspire_run.run()

Generating spot images...
Generating patch images...


Extracting features of spot images...


Experiment-wide values for mean threshold, etc calculated by MeasureImageQuality may be incorrect if the run is split into subsets of images.
Experiment-wide values for mean threshold, etc calculated by MeasureImageQuality may be incorrect if the run is split into subsets of images.
Times reported are CPU and Wall-clock times for each module
Thu Feb  9 10:19:38 2023: Image # 23, module Images # 1: CPU_time = 0.00 secs, Wall_time = 0.00 secs
Thu Feb  9 10:19:38 2023: Image # 23, module Metadata # 2: CPU_time = 0.00 secs, Wall_time = 0.00 secs
...


Thu Feb  9 10:19:56 2023: Image # 44, module ExportToSpreadsheet # 8: CPU_time = 0.00 secs, Wall_time = 0.00 secs
Thu Feb  9 10:19:56 2023: Image # 44, module CreateBatchFiles # 9: CPU_time = 0.00 secs, Wall_time = 0.00 secs


Extracting features of patch images...


Experiment-wide values for mean threshold, etc calculated by MeasureImageQuality may be incorrect if the run is split into subsets of images.
Given the large number of image sets, you may want to consider using ExportToDatabase as opposed to ExportToSpreadsheet.
Experiment-wide values for mean threshold, etc calculated by MeasureImageQuality may be incorrect if the run is split into subsets of images.
Times reported are CPU and Wall-clock times for each module
Thu Feb  9 10:20:09 2023: Image # 415, module Images # 1: CPU_time = 0.00 secs, Wall_time = 0.00 secs
Thu Feb  9 10:20:09 2023: Image # 415, module Metadata # 2: CPU_time = 0.00 secs, Wall_time = 0.00 secs
...


Thu Feb  9 10:24:09 2023: Image # 1863, module ExportToSpreadsheet # 8: CPU_time = 0.00 secs, Wall_time = 0.00 secs
Thu Feb  9 10:24:09 2023: Image # 1863, module CreateBatchFiles # 9: CPU_time = 0.00 secs, Wall_time = 0.00 secs


Solving OT...
It.  |Loss        |Relative loss|Absolute loss
------------------------------------------------
    0|1.897594e-01|0.000000e+00|0.000000e+00
    1|5.502879e-02|2.448366e+00|1.347306e-01
    2|5.225638e-02|5.305395e-02|2.772407e-03
    3|5.205054e-02|3.954582e-03|2.058381e-04
    4|5.201631e-02|6.582070e-04|3.423750e-05
    5|5.200428e-02|2.312918e-04|1.202816e-05
    6|5.199664e-02|1.468210e-04|7.634201e-06
    7|5.199051e-02|1.179768e-04|6.133674e-06
    8|5.198444e-02|1.167566e-04|6.069528e-06
    9|5.197854e-02|1.134739e-04|5.898208e-06
   10|5.197335e-02|9.982855e-05|5.188425e-06
Computing high resolution expression profiles...
